In [13]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
import os

In [14]:
time_int = 0
well_num = 12
experiment_date = "20240223"
root = "E:\\Nick\\Cole Trapnell's Lab Dropbox\\Nick Lammers\\Nick\\pecfin_dynamics\\fin_morphodynamics\\"


# load fin dataset
file_prefix = experiment_date + f"_well{well_num:04}"
point_prefix = file_prefix + f"_time{time_int:04}"
point_path = os.path.join(root, "built_data", "point_clouds", "feature_predictions", experiment_date, "")
point_path_out = os.path.join(root, "built_data", "point_clouds", "feature_predictions", experiment_date, "")
if not os.path.isdir(point_path_out):
    os.makedirs(point_path_out)
point_df = pd.read_csv(point_path + point_prefix + "_centroids.csv")

In [15]:
print(np.sum(point_df["fin_label_pd"]==0.5))

1001


In [16]:
label_dict=dict({0.25:"outlier", 0.75:"yolk", 0.5:"fin", 1.0:"body"})
string_labels = [label_dict[f] for f in list(point_df["fin_label_pd"])]
point_df["fin_labels_cat"] = string_labels
fig = px.scatter_3d(point_df, x="X", y="Y", z="Z", color="fin_labels_cat", opacity=1)

fig.update_layout(
    scene={
    "zaxis" : dict(autorange="reversed")
    }
)

fig.show()

In [17]:
fig = px.scatter_3d(point_df.loc[point_df["fin_labels_cat"]=="fin", :], 
                                 x="X", y="Y", z="Z", color="fin_labels_cat", opacity=0.5)

fig.update_layout(
    scene={
    "zaxis" : dict(autorange="reversed")
    }
)

fig.show()

In [18]:
import alphashape

xyz_fin = point_df.loc[point_df["fin_labels_cat"]=="fin", ["X", "Y", "Z"]].to_numpy()
points = xyz_fin / np.max(xyz_fin)
hull20 = alphashape.alphashape(points, 20)

In [19]:
plot_hull = hull20

tri_points = plot_hull.vertices[plot_hull.faces]

#extract the lists of x, y, z coordinates of the triangle vertices and connect them by a line
Xe = []
Ye = []
Ze = []
for T in tri_points:
    Xe.extend([T[k%3][0] for k in range(4)]+[ None])
    Ye.extend([T[k%3][1] for k in range(4)]+[ None])
    Ze.extend([T[k%3][2] for k in range(4)]+[ None])
    
       
#define the trace for triangle sides
fig = go.Figure()
lines = go.Scatter3d(
                   x=Xe,
                   y=Ye,
                   z=Ze,
                   mode='lines',
                   name='',
                   line=dict(color= 'rgb(70,70,70)', width=1)) 

fig.add_trace(go.Mesh3d(x=plot_hull.vertices[:, 0], y=plot_hull.vertices[:, 1],z=plot_hull.vertices[:, 2], 
                        opacity=0.5,
                               i=plot_hull.faces[:, 0], j=plot_hull.faces[:, 1], k=plot_hull.faces[:, 2]))

fig.add_trace(lines)
fig.update_layout(template="plotly")

fig.update_layout(
            scene=dict(
                zaxis=dict(autorange="reversed")))

fig.show()